# CMOR example with prototype CMIP7 tables/CVs

This code is a demonstration of the use of CMOR with the MIP tables in this repository and prototype CVs. This requires a conda environment with CMOR v3.13 installed (see cmor_environment.yml in this repository)

This example is up to date with the CMIP7 global attributes, including file and directory naming.

In [1]:
%time
import cmor
import numpy 
import json

CPU times: user 2 μs, sys: 1 μs, total: 3 μs
Wall time: 5.48 μs


The following is adapted from the code at https://github.com/PCMDI/cmor/blob/main/Test/test_cmor_CMIP7.py

Construct the input json file for CMOR


In [2]:
%time
DATASET_INFO = {
    "_AXIS_ENTRY_FILE": "tables/CMIP7_coordinate.json",
    "_FORMULA_VAR_FILE": "tables/CMIP7_formula_terms.json",
    "_cmip7_option": 1,
    "_controlled_vocabulary_file": "test/CMIP7-CV_for-cmor.json",  # SEPARATE TO MIP TABLES FOR TESTING ONLY
    "activity_id": "CMIP",
    "branch_method": "standard",
    "branch_time_in_child": 30.0,
    "branch_time_in_parent": 10800.0,
    "calendar": "360_day",
    "cv_version": "7.0.0.0",
    "drs_specs": "MIP-DRS7",
    "experiment_id": "historical",
    "forcing_index": "f3",
    "grid": "N96",
    "grid_label": "gn",
    "initialization_index": "i1",
    "institution_id": "PCMDI",
    "license_id": "CC BY 4.0",
    "nominal_resolution": "250 km",
    "outpath": ".",
    "parent_mip_era": "CMIP7",
    "parent_time_units": "days since 1850-01-01",
    "parent_activity_id": "CMIP",
    "parent_source_id": "PCMDI-test-1-0",
    "parent_experiment_id": "piControl",
    "parent_variant_label": "r1i1p1f3",
    "physics_index": "p1",
    "realization_index": "r9",
    "source_id": "PCMDI-test-1-0",
    "source_type": "AOGCM CHEM BGC",
    "tracking_prefix": "hdl:21.14107",
    "host_collection": "CMIP7",
    "frequency": "day",
    "region": "glb",
    "archive_id": "WCRP",
    "mip_era": "CMIP7",
}

# Adjustments for CVs table from esgvoc.
# This should end up being same as the above, but isn't at the moment.
DATASET_INFO["_controlled_vocabulary_file"] = "test/esgvoc-integration-cmor-cvs-table.json"
DATASET_INFO["grid_label"] = "g99"
DATASET_INFO["institution_id"] = "CCCma"
DATASET_INFO["license_id"] = "CC-BY-4-0"
DATASET_INFO["nominal_resolution"] = "100-km"
DATASET_INFO["parent_source_id"] = "CanESM6-MR"
DATASET_INFO["source_id"] = "CanESM6-MR"
DATASET_INFO["frequency"] = "mon"

with open('input.json', 'w') as fh:
    json.dump(DATASET_INFO, fh, indent=2)

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 5.25 μs


In [3]:
%time
cmor.setup(inpath="tables", netcdf_file_action=cmor.CMOR_REPLACE)

cmor.dataset_json('input.json')

tos = numpy.array([27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27
                    ])
tos.shape = (2, 3, 4)
lat = numpy.array([10, 20, 30])
lat_bnds = numpy.array([5, 15, 25, 35])
lon = numpy.array([0, 90, 180, 270])
lon_bnds = numpy.array([-45, 45,
                        135,
                        225,
                        315
                        ])
time = numpy.array([15.0, 45.0])
time_bnds = numpy.array([0.0, 30.0, 60.0])

CPU times: user 2 μs, sys: 1 μs, total: 3 μs
Wall time: 5.01 μs


In [4]:
%time
realm = "ocean"
cmor.load_table(f"CMIP7_{realm}.json")

CPU times: user 2 μs, sys: 1e+03 ns, total: 3 μs
Wall time: 5.01 μs


0

In [5]:
%time
cmorlat = cmor.axis("latitude",
                    coord_vals=lat,
                    cell_bounds=lat_bnds,
                    units="degrees_north")
cmorlon = cmor.axis("longitude",
                    coord_vals=lon,
                    cell_bounds=lon_bnds,
                    units="degrees_east")
cmortime = cmor.axis("time",
                    coord_vals=time,
                    cell_bounds=time_bnds,
                    units="days since 2018")
axes = [cmortime, cmorlat, cmorlon]

CPU times: user 1 μs, sys: 1 μs, total: 2 μs
Wall time: 5.01 μs


In [6]:
variable = "tos_tavg-u-hxy-sea"
cmortos = cmor.variable(variable, "degC", axes)


In [7]:
# manually apply cell measures
region = DATASET_INFO['region']
frequency = DATASET_INFO['frequency']
cell_measures_key = ".".join([realm] + variable.split("_") + [frequency, region.upper()])

with open('tables/CMIP7_cell_measures.json') as fh:
    cell_measures = json.load(fh)

# Check that cell_measures are valid ( option flags need to be manually replaced )
variable_cell_measures = cell_measures[cell_measures_key]
if variable_cell_measures in ["::OPT", "::MODEL"]:
    raise RuntimeError(f"found cell_measures '{variable_cell_measures}' which CMOR will not allow")

cmor.set_variable_attribute(cmortos, "cell_measures", "c", variable_cell_measures)

0

In [8]:
cmor.write(cmortos, tos)
filename = cmor.close(cmortos, file_name=True)
filename

'./MIP-DRS7//CMIP7//CMIP//CCCma//CanESM6-MR//historical//r9i1p1f3//glb//mon//tos//tavg-u-hxy-sea//g99//v20260106/tos_tavg-u-hxy-sea_mon_glb_g99_CanESM6-MR_historical_r9i1p1f3_201801-201802.nc'

Locate generated file

In [9]:
!find MIP-DRS7 -type f -name '*.nc'

MIP-DRS7/CMIP7/CMIP/CCCma/CanESM6-MR/historical/r9i1p1f3/glb/mon/tos/tavg-u-hxy-sea/g99/v20260106/tos_tavg-u-hxy-sea_mon_glb_g99_CanESM6-MR_historical_r9i1p1f3_201801-201802.nc


Dump header of generated file

In [10]:
!ncdump -h MIP-DRS7/CMIP7/CMIP/CCCma/CanESM6-MR/historical/r9i1p1f3/glb/mon/tos/tavg-u-hxy-sea/g99/v*/tos_tavg-u-hxy-sea_mon_glb_g99_CanESM6-MR_historical_r9i1p1f3_201801-201802.nc

netcdf tos_tavg-u-hxy-sea_mon_glb_g99_CanESM6-MR_historical_r9i1p1f3_201801-201802 {
dimensions:
	time = UNLIMITED ; // (2 currently)
	lat = 3 ;
	lon = 4 ;
	bnds = 2 ;
variables:
	double time(time) ;
		time:bounds = "time_bnds" ;
		time:units = "days since 2018" ;
		time:calendar = "360_day" ;
		time:axis = "T" ;
		time:long_name = "Time Intervals" ;
		time:standard_name = "time" ;
	double time_bnds(time, bnds) ;
	double lat(lat) ;
		lat:bounds = "lat_bnds" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
		lat:long_name = "Latitude" ;
		lat:standard_name = "latitude" ;
	double lat_bnds(lat, bnds) ;
	double lon(lon) ;
		lon:bounds = "lon_bnds" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
		lon:long_name = "Longitude" ;
		lon:standard_name = "longitude" ;
	double lon_bnds(lon, bnds) ;
	float tos(time, lat, lon) ;
		tos:standard_name = "sea_surface_temperature" ;
		tos:long_name = "Sea Surface Temperature" ;
		tos:units = "degC" ;
		tos:cell_methods = "area: mean where sea tim

In [11]:
# clean up
! rm -rf MIP-DRS7
#! rm -rf input.json

rm: cannot remove 'MIP-DRS7/CMIP7/CMIP/CCCma/CanESM6-MR/historical/r9i1p1f3/glb/mon/tos/tavg-u-hxy-sea/g99/v20260106/.nfs000000001879b9f700007bc2': Device or resource busy
